# tBLASTn in arthropods

In [2]:
import time
import os

In [3]:
clustered_seq = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-CD-HIT-updated.clstr"
fasta_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-HHMER-updated.fasta"

Extracting representative IDs from each cluster.

In [4]:
with open(clustered_seq, "r") as clusters:
    c = clusters.readlines()
rep_ID_lines = [e for e in c if e.endswith("*\n")]
rep_IDs = [e.split(", >")[1].split("...")[0] for e in rep_ID_lines]
rep_IDs

['QPN36968.1',
 'UQT02520.1',
 'QMP82367.1',
 'UVF62194.1',
 'QMP82195.1',
 'QMP82285.1',
 'QMP82388.1',
 'QMP82402.1',
 'QMP82403.1',
 'QMP82112.1',
 'QMP82168.1',
 'QMP82344.1',
 'QMP82362.1',
 'QMP82338.1',
 'QPZ88437.1',
 'UMO75733.1',
 'QED21504.1',
 'UPT53723.1',
 'AVR52572.1',
 'QMP82355.1',
 'AVR52567.1',
 'UUG74181.1',
 'UMO75734.1',
 'AJG39094.1',
 'QMP82147.1',
 'QPL15381.1',
 'QMP82248.1',
 'UMO75726.1',
 'QMP82204.1',
 'QPL15288.1',
 'QPL15340.1',
 'UMO75713.1',
 'UUG74233.1',
 'QPL15368.1',
 'UMO75705.1',
 'UMO75722.1',
 'QMP82272.1',
 'URQ09139.1',
 'QIJ70032.1',
 'QMP82229.1',
 'QWC36487.1',
 'UUG74184.1',
 'AJG39084.1',
 'UMO75717.1',
 'DAZ85673.1',
 'AJG39092.1',
 'AJG39088.1',
 'QDK54862.1',
 'QEM39320.1',
 'AJG39089.1',
 'QPZ88434.1',
 'AJG39086.1',
 'QKK82920.1',
 'QPL15369.1',
 'QPZ88431.1',
 'UUG74234.1',
 'ACY56282.1',
 'UMO75729.1',
 'UYL95486.1',
 'ASR92124.1',
 'UUT42596.1',
 'QPN36942.1',
 'URY50684.1',
 'UUT42625.1',
 'AIY25030.1',
 'QOE76794.1',
 'DAZ91106

In [6]:
len(rep_IDs)

109

Moving selected sequences to a new directory.

In [8]:
import shutil

source_folder = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1/"
destination_folder = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/PB1-clusters/"

for file in rep_IDs:
    source = source_folder + f"{file}.fasta"
    destination = destination_folder + f"{file}.fasta"
    shutil.copy(source, destination)

# Building nt database for local BLAST

If the database is alreade built, skip to [tBLASTn](#running-local-blast).

Following https://bioinf.shenwei.me/taxonkit/tutorial/#extract-all-protein-sequences-of-specific-taxons-from-the-ncbi-nr-database tutorial

Steps:

### 1. Setting up taxonkit:
1.1 Downloаd taxonkit

1.2 Downloading and decompressing taxdump.tar.gz: ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz (as recommended https://bioinf.shenwei.me/taxonkit/download/)


In [ ]:
%%bash
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz' --output ./taxdump.tar.gz

In [ ]:
%%bash
tar -zxvf taxdump.tar.gz

1.3 Copying some of the taxdump files to `$HOME/.taxonkit` directory.

In [ ]:
%%bash
cp names.dmp nodes.dmp delnodes.dmp merged.dmp $HOME/.taxonkit

### 2. Writing Arthropoda IDs to text file.

In [ ]:
%%bash
id=6656
taxonkit list --ids $id --indent "" > $id.taxid.txt
wc -l $id.taxid.txt

  925155 6656.taxid.txt


### 3. Retrieving target accessions:

3.1 Downloading `nucl_gb.accession2taxid.gz` file for nt database (or `prot.accession2taxid.gz` for nr db) from NCBI server: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/

Note: specify the name of the output file in curl command (the name of directory isn't enough).

In [ ]:
%%bash
# nt database
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz' --output /Volumes/WD_BLACK/Brigita/nt/nucl_gb.accession2taxid.gz

In [ ]:
%%bash
# nr database
curl 'ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz' --output /Volumes/EXTERNAL/Orthomyxoviridae/prot.accession2taxid.gz

3.2 Retrieving target accessions (taxid -> accession numbers).

In [4]:
%%bash
# nt database
source ~/.bashrc
pigz -dc /Volumes/WD_BLACK/Brigita/nt/nucl_gb.accession2taxid.gz \
    | csvtk grep -t -f taxid -P /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.txids \
    | csvtk cut -t -f accession.version,taxid \
    | sed 1d \
    > /Volumes/WD_BLACK/Brigita/nt/6656.acc2taxid.txt

cut -f 1 /Volumes/WD_BLACK/Brigita/nt/6656.acc2taxid.txt > /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt

wc -l /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt

 34241660 /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt


In [ ]:
%%bash
# nr database
pigz -dc /Volumes/EXTERNAL/Orthomyxoviridae/prot.accession2taxid.gz \
    | csvtk grep -t -f taxid -P /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.taxid.txt \
    | csvtk cut -t -f accession.version,taxid \
    | sed 1d \
    > /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc2taxid.txt

cut -f 1 /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc2taxid.txt > /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt

wc -l /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt

 19035895 /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/6656.acc.txt


### 4. Retrieving FASTA sequences from pre-formated blastdb.

4.1 Downloading databases:

In [ ]:
%%bash 
# nt database
curl -O "ftp://ftp.ncbi.nih.gov/blast/db/nt_euk.[00-100].tar.gz" --output-dir /Volumes/WD_BLACK/Brigita/nt/pre_nt

In [ ]:
%%bash
ls nt_euk.*.tar.gz | xargs -I % tar -zxv -f % -C /Volumes/WD_BLACK/Brigita/nt/extracted

In [ ]:
%%bash
# nr database
curl -O "ftp://ftp.ncbi.nih.gov/blast/db/nr.[00-100].tar.gz" --output-dir /Volumes/EXTERNAL/Orthomyxoviridae/blast

In [ ]:
%%bash
ls nr.*.tar.gz | xargs -I % tar -zxv -f % -C /Volumes/EXTERNAL/Orthomyxoviridae/blast/extracted/

Exporting nt_euk.fa (nr.fa) from pre-formated blastdb. 

In [ ]:
%%bash
blastdbcmd -db /Volumes/WD_BLACK/Brigita/nt/extracted/nt_euk -dbtype nucl -entry all -outfmt "%f" -out - | pigz -c > /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.fa.gz

In [ ]:
%%bash
cd /Volumes/EXTERNAL/Orthomyxoviridae/blast/nr
blastdbcmd -db nr -dbtype prot -entry all -outfmt "%f" -out - | pigz -c > /Volumes/EXTERNAL/Orthomyxoviridae/nr.fa.gz

Installing seqkit.

The script below was written as a text file and run from a command line: bash filter-sequence.sh 

Extracting *Arthropoda* sequences from the entire database.

In [ ]:
%%bash
# nt database
time cat <(echo) <(pigz -dc /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.fa.gz) \
    | perl -e 'BEGIN{ $/ = "\n>"; <>; } while(<>){s/>$//;  $i = index $_, "\n"; $h = substr $_, 0, $i; $s = substr $_, $i+1; if ($h !~ />/) { print ">$_"; next; }; $h = ">$h"; while($h =~ />([^ ]+ .+?) ?(?=>|$)/g){ $h1 = $1; $h1 =~ s/^\W+//; print ">$h1\n$s";} } ' \
    | seqkit grep -f /Volumes/WD_BLACK/Brigita/nt/6656.acc.txt -o /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa.gz

In [ ]:
%%bash
#nr database
time cat <(echo) <(pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.fa.gz) \
    | perl -e 'BEGIN{ $/ = "\n>"; <>; } while(<>){s/>$//;  $i = index $_, "\n"; $h = substr $_, 0, $i; $s = substr $_, $i+1; if ($h !~ />/) { print ">$_"; next; }; $h = ">$h"; while($h =~ />([^ ]+ .+?) ?(?=>|$)/g){ $h1 = $1; $h1 =~ s/^\W+//; print ">$h1\n$s";} } ' \
    | seqkit grep -f /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/6656.acc.txt -o /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz

Counting sequences.

In [ ]:
%%bash
wc -l /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/6656.acc.txt
pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz | grep '^>' -c

10746263


Formatting BLAST database.

In [ ]:
%%bash
# nt database
pigz -dc /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa.gz > /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa

In [ ]:
%%bash
# nt database
makeblastdb -parse_seqids -in /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa -dbtype nucl -out /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656

Output:

>Building a new DB, current time: 09/06/2023 17:52:16\
>New DB name:   /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656\
>New DB title:  /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656.fa\
>Sequence type: Nucleotide\
>Keep MBits: T\
>Maximum file size: 3000000000B\
>Adding sequences from FASTA; added 17068454 sequences in 9733.49 seconds.

In [9]:
%%bash
# nr database
pigz -dc /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa.gz > /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa

In [10]:
%%bash
# nr database
makeblastdb -parse_seqids -in /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa -dbtype prot -out /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656



Building a new DB, current time: 08/16/2023 11:22:29
New DB name:   /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656
New DB title:  /Volumes/WD_BLACK/Brigita/Orthomyxoviridae/nr.6656.fa
Sequence type: Protein
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 10746263 sequences in 509.398 seconds.




# Running local BLAST.

Database (-db): nt/6656/nt_euk.6656\
E-value (-evalue): 1e-6\
CPUs (-num_threads): 8\
Number of hits (-max_target_seqs): 1000\
Output format (-outfmt): 5 (output is written in XML)

In [ ]:
%%bash
# BLASTing 20 sequences
time tblastn -db nt/6656/nt_euk.6656 -query PB1-20seq.fasta -out PB1-20tblastn.xml -evalue 1e-6 -num_threads 8 -max_target_seqs 1000 -outfmt 5

In [ ]:
%%bash
#test run
tblastn -db /Volumes/WD_BLACK/Brigita/nt/6656/nt_euk.6656 -query /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/blast-test.fasta -out /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/blast-result2 -evalue 1e-6 -num_threads 4 -max_target_seqs 1000 -outfmt 5

Another way to download nt database:

In [ ]:
%%bash 
update_blastdb.pl --decompress nt

# Analyzing hits

In [7]:
import pandas as pd
from Bio import SearchIO

Creating list with accession numbers of queries (viral PB1 sequences).

In [14]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
BLAST_results_89 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-89tblastn.xml", "blast-xml")
PB1 = []

for query in BLAST_results_20:
    PB1.append(query.id)

for query in BLAST_results_89:
    PB1.append(query.id)

In [15]:
len(PB1)

109

In [41]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
hits_total = []
for query in BLAST_results_20:
    print(query.id)
    hits = []
    for hit in query:
        hits.append(hit.id)
    hits_total.append(len(hits))
    print(f"Number of hits: ", len(hits))
    print("--------------------------")

print(sum(hits_total))

AAA42968.1
Number of hits:  259
--------------------------
AAC25959.1
Number of hits:  249
--------------------------
AAL67962.1
Number of hits:  13
--------------------------
AAU94857.1
Number of hits:  306
--------------------------
ACY56282.1
Number of hits:  640
--------------------------
AED98371.1
Number of hits:  262
--------------------------
AIY25030.1
Number of hits:  630
--------------------------
AJG39084.1
Number of hits:  646
--------------------------
AJG39085.1
Number of hits:  588
--------------------------
AJG39086.1
Number of hits:  672
--------------------------
AJG39088.1
Number of hits:  673
--------------------------
AJG39089.1
Number of hits:  672
--------------------------
AJG39092.1
Number of hits:  651
--------------------------
AJG39094.1
Number of hits:  646
--------------------------
AJG39096.1
Number of hits:  685
--------------------------
ALJ78713.1
Number of hits:  268
--------------------------
APP91612.1
Number of hits:  269
-------------------------

In [42]:
BLAST_results_20 = SearchIO.parse("/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/PB1-20tblastn.xml", "blast-xml")
count = 0
for query in BLAST_results_20:
    print(query.id)
    for hit in query:
        count += 1
        hsps = []
        for hsp in hit:
            hsps.append(hsp)
        print(f"{count} Number of HSPs for {hit.id}: ", len(hsps))    
    print("--------------------------")

AAA42968.1
1 Number of HSPs for emb|OW052077.1|:  16
2 Number of HSPs for emb|OX421934.1|:  5
3 Number of HSPs for emb|OX411715.1|:  8
4 Number of HSPs for emb|OX382229.1|:  3
5 Number of HSPs for emb|OU342663.1|:  1
6 Number of HSPs for emb|OY292434.1|:  1
7 Number of HSPs for emb|LR990212.1|:  2
8 Number of HSPs for emb|OV656855.1|:  4
9 Number of HSPs for emb|OV121058.1|:  2
10 Number of HSPs for emb|FR990001.1|:  7
11 Number of HSPs for emb|OU744810.1|:  10
12 Number of HSPs for emb|OW388277.2|:  2
13 Number of HSPs for emb|OU426947.1|:  1
14 Number of HSPs for emb|OX424458.1|:  4
15 Number of HSPs for emb|OU026114.1|:  2
16 Number of HSPs for emb|OU974029.1|:  1
17 Number of HSPs for emb|OX419599.1|:  2
18 Number of HSPs for emb|OX421465.1|:  1
19 Number of HSPs for emb|OX382355.1|:  2
20 Number of HSPs for emb|OW203721.1|:  3
21 Number of HSPs for emb|OY560206.1|:  1
22 Number of HSPs for emb|OW971904.1|:  1
23 Number of HSPs for emb|OW052072.1|:  1
24 Number of HSPs for emb|LR99